# Notebook
version 15h55 Tu Jan 21, sent by Raphael Zaghroun

## Imports and Setup

In [1]:
import sys 
print(sys.executable)

/Users/trenaudie/Documents/cours/mines_3a/aneurysms/ButterflyAnevrisme/venv/bin/python


In [2]:
import torch 
def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
print(CUDA_version)


None


In [3]:
if CUDA_version is not None:
    CUDA = format_cuda_version(CUDA_version)
    print(CUDA, TORCH)
else:
    CUDA = 'cpu'
    print(CUDA, TORCH)
# from google.colab import drive
# drive.mount('/content/drive')


cpu 2.5.1


In [4]:
import tensorboard

In [5]:
tensorboard.__version__

'2.17.1'

In [6]:
# Standard library imports
import enum
import gc
import os
import os.path as osp
import shutil
import sys
from itertools import product
from typing import List
# Third-party library imports
import meshio
import numpy as np
import torch
from loguru import logger
from torch.nn.modules.loss import _Loss
from torch.utils.data import DataLoader, Dataset as BaseDataset
from torch_scatter import scatter_add
import torch.nn as nn
from torch_geometric.data import Data
from tqdm import tqdm


## Dataset and Dataloader

In [7]:

class Dataset(BaseDataset):
    def __init__(
        self,
        folder_path: str,
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.folder_path = folder_path
        self.files = os.listdir(folder_path)
        self.files = [file for file in self.files if file.endswith(".xdmf")]
        self.files.sort()
        self.len_time = 79
        self.number_files = len(self.files) * self.len_time
        self.encode_id = {i*self.len_time+t:(i,t) for t,i in product(range(self.len_time),range(len(self.files)))}

    def __len__(self):
      return self.number_files

    def __getitem__(self,id):
        i,t = self.encode_id[id]
        meshes = self.xdmf_to_meshes(self.folder_path+self.files[i],t)
        mesh = meshes[0]

        #Get data from mesh
        data, pos, edges, edges_attr = self.mesh_to_graph_data(mesh,t)

        #Get speed for t+1 mesh
        next_t_mesh = meshes[1]
        next_data = self.get_speed_data(next_t_mesh,t+1)
        del meshes

        #Structure the information
        current_graph_data = {
                              "x":data,
                              "pos":pos,
                              "edge_index":edges,
                              "edge_attr":edges_attr,
                              "y":next_data[:,:-2],
                              }

        # graph_data = Data(x=current_graph_data['x'],
        #pos=current_graph_data['pos'],
        #                         edge_index=current_graph_data['edge_index'],
        #                         edge_attr=current_graph_data['edge_attr'],
        #                         y=current_graph_data['y'])

        return current_graph_data

    def get_speed_data(self,mesh,t):
        time_array = np.full(mesh.point_data['Pression'][:,None].shape, fill_value=t*1e-2)
        data = torch.from_numpy(np.concatenate([mesh.point_data['Vitesse'],
                                                  mesh.point_data['Pression'][:,None],
                                                  time_array],axis=1))
        return data

    def mesh_to_graph_data(self,mesh,t):
        node_edges = []
        edges_attr_ = []
        for tetra in mesh.cells_dict['tetra']:
            for node,neighbor in product(tetra,tetra):
                if node != neighbor:
                    node_edges.append([node,neighbor])
                    edges_attr_.append(mesh.points[neighbor]-mesh.points[node])
        edges = torch.from_numpy(np.array(node_edges).T)
        edges_attr = torch.from_numpy(np.array(edges_attr_))
        pos = torch.from_numpy(mesh.points)
        wall_labels = self.classify_vertices(mesh, "Vitesse")  # Assuming classify_vertices returns 0 for wall, 1 for others
        wall_labels_tensor = torch.tensor(wall_labels).unsqueeze(1)  # Convert to tensor and add dimension
        data = self.get_speed_data(mesh,t)
        data = torch.cat([data, wall_labels_tensor], dim=1)  # Concatenate with data
        return data, pos, edges, edges_attr

    @staticmethod
    def classify_vertices(mesh: meshio.Mesh, velocity_key: str = "Vitesse") -> np.ndarray:
        """
        Classify each vertex of the mesh as being on the wall or in the flow.

        Parameters:
            mesh: The mesh object containing point data.
            velocity_key: The key for the velocity data in the mesh point_data.

        Returns:
            A numpy array of labels (0 for wall, 1 for flow).
        """
        if velocity_key not in mesh.point_data:
            raise ValueError(f"Velocity data key '{velocity_key}' not found in mesh point_data.")

        velocities = np.array(mesh.point_data[velocity_key])  # Shape: (num_points, 3)
        speed_norm = np.linalg.norm(velocities, axis=1)  # Compute the norm of velocity for each vertex
        labels = np.where(speed_norm <= 1e-8, 0, 1)  # 0 for wall, 1 for flow
        return labels

    @staticmethod
    def xdmf_to_meshes(xdmf_file_path: str, t) -> List[meshio.Mesh]:
        """
        Opens an XDMF archive file, and extract a data mesh object for every timestep.

        xdmf_file_path: path to the .xdmf file.
        Returns: list of data mesh objects.
        """

        reader = meshio.xdmf.TimeSeriesReader(xdmf_file_path)
        points, cells = reader.read_points_cells()
        meshes = []

        # Extracting the meshes from the archive
        for i in [t,t+1]:
            # Depending on meshio version, the function read_data may return 3 or 4 values.
            try:
                time, point_data, cell_data, _ = reader.read_data(i)
            except ValueError:
                time, point_data, cell_data = reader.read_data(i)
            mesh = meshio.Mesh(points, cells, point_data=point_data, cell_data=cell_data)
            meshes.append(mesh)
        print(f"Loaded {len(meshes)} timesteps from {xdmf_file_path.split('/')[-1]}\n")
        return meshes


folder_path = '/content/drive/MyDrive/IDSC/4Students_AnXplore03/'

# dataset = Dataset(folder_path)
# train_loader = DataLoader(
#     dataset=dataset,
#     batch_size=1,
#     shuffle=True,
#     num_workers=2,
# )

## Model

In [8]:
def build_mlp(
    in_size: int,
    hidden_size: int,
    out_size: int,
    nb_of_layers: int = 4,
    lay_norm: bool = True,
) -> nn.Module:
    """
    Builds a Multilayer Perceptron (MLP) using PyTorch.

    Parameters:
        - in_size (int): The size of the input layer.
        - hidden_size (int): The size of the hidden layers.
        - out_size (int): The size of the output layer.
        - nb_of_layers (int, optional): The number of layers in the MLP, including the input and output layers. Defaults to 4.

    Returns:
        - nn.Module: The constructed MLP model.
    """
    # Initialize the model with the first layer.
    layers = []
    layers.append(nn.Linear(in_size,hidden_size))
    layers.append(nn.ReLU())

    if lay_norm:
      layers.append(nn.LayerNorm(hidden_size))

    for _ in range(nb_of_layers - 2):
      layers.append(nn.Linear(hidden_size,hidden_size))
      layers.append(nn.ReLU())

      if lay_norm:
        layers.append(nn.LayerNorm(hidden_size))

    # Add the output layer
      layers.append(nn.Linear(hidden_size,out_size))

    # Construct the model using the specified layers.
    module = nn.Sequential(*layers)

    return module

class EdgeBlock(nn.Module):
    """A block that updates the attributes of the edges in a graph based on the features of the
    sending and receiving nodes, as well as the original edge attributes.

    Attributes:
        model_fn (callable): A function to update edge attributes.
    """

    def __init__(self, model_fn=None):

        super(EdgeBlock, self).__init__()
        self._model_fn = model_fn

    def forward(self, graph):
        """Forward pass of the EdgeBlock.

        Args:
            graph (Data): A graph containing node attributes, edge indices, and edge attributes.

        Returns:
            Data: An updated graph with new edge attributes.
        """
        edge_inputs = torch.concat(
            [
                graph.edge_attr,
                graph.x[graph.edge_index[0]],
                graph.x[graph.edge_index[1]]
            ], dim=1
        )
        # print(f'edge_inputs {edge_inputs.shape}')

        edge_attr_ = self._model_fn(edge_inputs)

        return Data(
                x=graph.x, edge_attr=edge_attr_, edge_index=graph.edge_index, pos=graph.pos
            )


class NodeBlock(nn.Module):
    """A block that updates the attributes of the nodes in a graph based on the aggregated features
    of the incoming edges and the original node attributes.

    Attributes:
        model_fn (callable): A function to update node attributes.
    """

    def __init__(self, model_fn=None):

        super(NodeBlock, self).__init__()

        self._model_fn = model_fn

    def forward(self, graph):
        """Forward pass of the NodeBlock.

        Args:
            graph (Data): A graph containing node attributes, edge indices, and edge attributes.

        Returns:
            Data: An updated graph with new node attributes.
        """
        edge_attr = graph.edge_attr
        receivers_indx = graph.edge_index[1]
        agrr_edge_features = scatter_add(
            edge_attr, receivers_indx, dim=0, dim_size=graph.num_nodes
        )
        node_inputs = torch.cat(
            [graph.x, agrr_edge_features], dim=-1
        )

        x_ = self._model_fn(node_inputs)

        return Data(
                x=x_, edge_attr=graph.edge_attr, edge_index=graph.edge_index, pos=graph.pos
            )

class GraphNetBlock(nn.Module):
    """A block that sequentially applies an EdgeBlock and a NodeBlock to update the attributes of
    both edges and nodes in a graph.

    Attributes:
        edge_block (EdgeBlock): The block to update edge attributes.
        node_block (NodeBlock): The block to update node attributes.
    """

    def __init__(
        self,
        hidden_size=128,
        use_batch=False,
        use_gated_mlp=False,
        use_gated_lstm=False,
        use_gated_mha=False,
    ):

        super(GraphNetBlock, self).__init__()

        edge_input_dim = 3*hidden_size #3*128=384
        node_input_dim = 2*hidden_size #2*128=256

        self.edge_block = EdgeBlock(model_fn=build_mlp(
            in_size=edge_input_dim,
            hidden_size=hidden_size,
            out_size=hidden_size,
        )) #
        self.node_block = NodeBlock(
            model_fn=build_mlp(
                in_size=node_input_dim,
                hidden_size=hidden_size,
                out_size=hidden_size,
            )
        ) #

    def _apply_sub_block(self, graph):
        graph = self.edge_block(graph)
        return self.node_block(graph)

    def forward(self, graph):

        graph_last = graph.clone()
        graph = self._apply_sub_block(graph)

        edge_attr = graph_last.edge_attr + graph.edge_attr
        x = graph_last.x + graph.x

        return Data(
                x=x, edge_attr=edge_attr, edge_index=graph.edge_index, pos=graph.pos
            )

## Training Utils

In [11]:


class Epoch:
    def __init__(
        self,
        model,
        loss,
        stage_name,
        parameters,
        device="cpu",
        verbose=True,
        starting_step=0,
    ):
        self.model = model
        self.loss = loss
        self.verbose = verbose
        self.device = device
        self.parameters = parameters
        self.step = 0
        self._to_device()
        self.stage_name = stage_name
        self.full_batch_graph = []
        self.starting_step = starting_step

    def _to_device(self):
        self.model.to(self.device)
        self.loss.to(self.device)

    def _format_logs(self, logs):
        str_logs = ["{} - {:.4}".format(k, v) for k, v in logs.items()]
        s = ", ".join(str_logs)
        return s

    def batch_update(self, x, y):
        raise NotImplementedError

    def on_epoch_start(self):
        pass

    def run(self, dataloader, writer=None, model_save_dir="checkpoint/simulator.pth"):

        self.on_epoch_start()

        logs = {}
        loss_meter = AverageValueMeter()

        with tqdm(
            dataloader,
            desc=self.stage_name,
            file=sys.stdout,
            disable=not (self.verbose),
        ) as iterator:
            for graph_data in iterator:
                for indx in range(1):
                    #TODO: check if we need this processing or not because it may already be the case
                    input_graph = Data(
                        x=graph_data["x"][indx],
                        pos=graph_data["pos"][indx],
                        edge_index=graph_data["edge_index"][indx],
                        edge_attr=graph_data.get("edge_attr", [None])[indx],
                        y=graph_data["y"][indx],
                    ).to(self.device)
                    # input_graph = graph_data.to(self.device)

                    self.full_batch_graph.append(input_graph)

                if len(self.full_batch_graph) % self.model.batch_size == 0:

                    loss = self.batch_update(self.full_batch_graph, writer)

                    # update loss logs
                    loss_value = loss.cpu().detach().numpy()
                    loss_meter.add(loss_value)
                    loss_logs = {self.loss.__name__: loss_meter.mean}
                    logs.update(loss_logs)

                    if self.model.training:
                        writer.add_scalar(
                            "Loss/train/value_per_step",
                            loss_value,
                            self.step + self.starting_step,
                        )

                    else:
                        writer.add_scalar(
                            "Loss/test/value_per_step",
                            loss_value,
                            self.step + self.starting_step,
                        )

                    if self.step % 200 == 0:
                        self.model.save_checkpoint(model_save_dir)
                        writer.flush()

                    self.step += 1
                    self.full_batch_graph = []

                    if self.verbose:
                        s = self._format_logs(logs)
                        iterator.set_postfix_str(s)

        return loss_meter.mean


class TrainEpoch(Epoch):
    def __init__(
        self,
        model,
        loss,
        parameters,
        optimizer,
        device="cpu",
        verbose=True,
        starting_step=0,
        use_sub_graph=False,
    ):
        super().__init__(
            model=model,
            loss=loss,
            stage_name="train",
            parameters=parameters,
            device=device,
            verbose=verbose,
            starting_step=starting_step,
        )
        self.optimizer = optimizer
        self.use_sub_graph = use_sub_graph

    def on_epoch_start(self):
        self.model.train()

    def batch_update(self, batch_graph, writer):
        self.optimizer.zero_grad()
        loss = 0
        #TODO: check that batch_graph is either a list of graphs of a list of one list of graphs
        for graph in batch_graph:
            node_type = graph['x'][:, self.model.node_type_index]
            network_output, target_delta_normalized = self.model(graph)
            loss += self.loss(
                target_delta_normalized,
                network_output,
                node_type,
            )

        loss /= len(batch_graph)
        loss.backward()
        max_norm = 10.0
        nn.utils.clip_grad_norm_(self.model.parameters(), max_norm)
        self.optimizer.step()

        return loss



In [12]:
class Meter(object):
    """Meters provide a way to keep track of important statistics in an online manner.
    This class is abstract, but provides a standard interface for all meters to follow.
    """

    def reset(self):
        """Reset the meter to default settings."""

    def add(self, value):
        """Log a new value to the meter
        Args:
            value: Next result to include.
        """

    def value(self):
        """Get the value of the meter in the current state."""

class AverageValueMeter(Meter):
    def __init__(self):
        super(AverageValueMeter, self).__init__()
        self.reset()
        self.val = 0

    def add(self, value, n=1):
        self.val = value
        self.sum += value
        self.var += value * value
        self.n += n

        if self.n == 0:
            self.mean, self.std = np.nan, np.nan
        elif self.n == 1:
            self.mean = 0.0 + self.sum  # This is to force a copy in torch/numpy
            self.std = np.inf
            self.mean_old = self.mean
            self.m_s = 0.0
        else:
            self.mean = self.mean_old + (value - n * self.mean_old) / float(self.n)
            self.m_s += (value - self.mean_old) * (value - self.mean)
            self.mean_old = self.mean
            self.std = np.sqrt(self.m_s / (self.n - 1.0))

    def value(self):
        return self.mean, self.std

    def reset(self):
        self.n = 0
        self.sum = 0.0
        self.var = 0.0
        self.val = 0.0
        self.mean = np.nan
        self.mean_old = 0.0
        self.m_s = 0.0
        self.std = np.nan

class L2Loss(_Loss):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    @property
    def __name__(self):
        return "MSE"

    def forward(
        self, target_speed, network_output, node_type
    ):
        "Computes L2 loss on velocity, with respect to the noise"
        mask = (node_type == 1)
        errors = (target_speed[mask] - network_output[mask]) ** 2
        return torch.mean(errors)


## Objects

In [13]:
from torch.utils.tensorboard import SummaryWriter

In [14]:


#from DataLoader import Dataset
#from EncoderDecoder import EncodeProcessDecode
#from Utils import L2Loss, Simulator
#from Epoch import TrainEpoch

writer = SummaryWriter("tensorboard")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
folder_path = 'data/'
dataset = Dataset(folder_path)

train_loader = DataLoader(
    dataset=dataset,
    batch_size=1,
    shuffle=True,
    num_workers=0,
    #multiprocessing_context='spawn'
)

model = EncodeProcessDecode(
    node_input_size=6,
    edge_input_size=3,
    message_passing_num=15,
    hidden_size=128,
    output_size=3,
) #
loss = L2Loss() #
simulator = Simulator(
    node_input_size=6,
    edge_input_size=3,
    output_size=3,
    feature_index_start=0,
    feature_index_end=4,
    output_index_start=0,
    output_index_end=3,
    node_type_index=5,
    batch_size=1,
    model=model,
    device=device,
    model_dir="checkpoint/simulator.pth",
    time_index=4
) #
optimizer = torch.optim.Adam(simulator.parameters(), lr=0.0001)


In [15]:
for batch in train_loader:
    print(batch)
    break

Loaded 2 timesteps from AllFields_Resultats_MESH_194.xdmf

{'x': tensor([[[-3.2071e-14,  1.0350e-14,  3.1645e-15,  5.3006e+02,  1.0000e-02,
           0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  5.2887e+02,  1.0000e-02,
           0.0000e+00],
         [ 3.8355e-13, -1.3933e-13,  1.0816e-14,  5.2624e+02,  1.0000e-02,
           0.0000e+00],
         ...,
         [ 2.4687e+02,  1.4630e+01,  4.1076e+01,  5.9381e+02,  1.0000e-02,
           1.0000e+00],
         [-3.0978e+01, -3.2599e+01,  4.9582e+00,  6.0487e+02,  1.0000e-02,
           1.0000e+00],
         [ 2.8117e+02, -1.9362e+02,  9.7901e+00,  5.1412e+02,  1.0000e-02,
           1.0000e+00]]], dtype=torch.float64), 'pos': tensor([[[ 1.0591e+00,  4.0783e+00,  4.9301e-01],
         [ 1.1285e+00,  4.0054e+00,  2.1423e-01],
         [ 1.2922e+00,  3.9372e+00, -2.3159e-02],
         ...,
         [-3.3884e+00,  5.8008e+00,  1.6536e+00],
         [ 3.8517e-02,  1.1090e+01,  9.3458e-04],
         [ 3.5942e+00,  4.3794e+

In [17]:
batch.keys()
# def build_graph_from_batch
simulator.batch_size
graph_data = batch
indx = 0
input_graph = Data(
            x=graph_data["x"][indx],
            pos=graph_data["pos"][indx],
            edge_index=graph_data["edge_index"][indx],
            edge_attr=graph_data.get("edge_attr", [None])[indx],
            y=graph_data["y"][indx],
    ).to(simulator.device)
graph = input_graph
x = graph.x
pos = graph.pos
edge_index = graph.edge_index
edge_attr = graph.edge_attr
y = graph.y
print(x.shape)
print(pos.shape)
print(edge_index.shape)
print(edge_attr.shape)
print(y.shape)

torch.Size([11704, 6])
torch.Size([11704, 3])
torch.Size([2, 682788])
torch.Size([682788, 3])
torch.Size([11704, 3])


In [18]:
encoder = model.encoder
encoder
encoded_graph = encoder(graph)

In [42]:

encoded_graph

Data(x=[11732, 128], edge_index=[2, 684396], edge_attr=[684396, 128], y=[11732, 3], pos=[11732, 3])

In [19]:

edge_block = model.processer_list[0].edge_block
edge_block


EdgeBlock(
  (_model_fn): Sequential(
    (0): Linear(in_features=384, out_features=128, bias=True)
    (1): ReLU()
    (2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): ReLU()
    (5): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (6): Linear(in_features=128, out_features=128, bias=True)
    (7): ReLU()
    (8): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (9): Linear(in_features=128, out_features=128, bias=True)
  )
)

In [20]:
after_edge = edge_block(encoded_graph)
after_edge

edge_inputs torch.Size([682788, 384])


Data(x=[11704, 128], edge_index=[2, 682788], edge_attr=[682788, 128], pos=[11704, 3])

In [22]:
graph = graph.to("cpu")
simulator = simulator.to("cpu")
simulator.device = "cpu"
encoded_graph = simulator.encoder(graph)
encoded_graph

device(type='cpu')

In [11]:

batch.keys()
#dict_keys(['x', 'pos', 'edge_index', 'edge_attr', 'y'])
simulator._build_input_graph(batch)



NameError: name 'batch' is not defined

In [27]:

x, pos, edge_index, edge_attr, y = batch['x'], batch['pos'], batch['edge_index'], batch['edge_attr'], batch['y']
x.shape #features
print(x.shape)
print(pos.shape) #coordinates, should not move
print(edge_index.shape) #edges
print(edge_attr.shape) #edge features
print(y.shape) #target


torch.Size([1, 11539, 6])
torch.Size([1, 11539, 3])
torch.Size([1, 2, 673452])
torch.Size([1, 673452, 3])
torch.Size([1, 11539, 3])


In [ ]:

input_graph = Data(
        x=graph_data["x"][indx],
        pos=graph_data["pos"][indx],
        edge_index=graph_data["edge_index"][indx],
        edge_attr=graph_data.get("edge_attr", [None])[indx],
        y=graph_data["y"][indx],
    ).to(self.device)


In [ ]:

print("now onto training !")

In [ ]:


train_epoch = TrainEpoch(
    model=simulator,
    loss=loss,
    optimizer=optimizer,
    parameters={},
    device=device,
    verbose=True,
    starting_step=0,
    use_sub_graph=False,
)  #

for i in range(0, 10):
    print("\nEpoch: {}".format(i))
    print("=== Training ===")
    train_loss = train_epoch.run(train_loader, writer, "model.pth")

    writer.add_scalar("Loss/train/mean_value_per_epoch", train_loss, i)
    writer.flush()
    writer.close()


Epoch: 0
=== Training ===
train:   0%|          | 0/8137 [00:00<?, ?it/s]Loaded 2 timesteps from AllFields_Resultats_MESH_42-1.xdmf

edge_inputs torch.Size([697272, 384])
edge_inputs torch.Size([697272, 384])
edge_inputs torch.Size([697272, 384])


In [ ]:
torch.cuda.empty_cache()